In [8]:
from code_tokenizer import *

# code = """
# posicao.Trigger == "DEACTIVATION" || posicao.Trigger == "POWER_LOSS"
# """

code = """
my_list = [1, 2, 3, 4, 5]
min = my_list[0]
for(i = 1;i < my_list.length; i++) {
    if(my_list[i] < min) {
        min = my_list[i]
    }
}
console.log(min)
"""

print(color_by_char_type(code))

parsed = math_layer(code)

def scan(data):
    if(type(data) == list and len(data) > 0):
        for item in data:
            scan(item)
scan(parsed)
print(parsed)


a = 1
if(a == 1) {
    a = 2
}

['assign', [['word', 'a'], ['number', '1']]]
['word', 'if', ['attached_group', ['operation', '==', [['word', 'a'], ['number', '1']]]], ['attached_block', ['assign', [['word', 'a'], ['number', '2']]]]]
